In [1]:
import os
import requests

from dotenv import load_dotenv

In [2]:
load_dotenv()
MEGAPI_URL = os.getenv("MEGAPI_URL")
MEGAPI_USERNAME = os.getenv("MEGAPI_USERNAME")
MEGAPI_PASSWORD = os.getenv("MEGAPI_PASSWORD")

In [3]:
def get_token():
    url = f"{MEGAPI_URL}/auth/token"
    data = {
        "grant_type": "",
        "username": MEGAPI_USERNAME,
        "password": MEGAPI_PASSWORD,
    }
    response = requests.post(url, data=data)
    return response.json()['access_token']


def get_n_examples_of_genre(genre, n):
    url = f"{MEGAPI_URL}/music_library/random-genre-examples/"
    headers = { "Authorization": f"Bearer {get_token()}" }
    data = {
        "genre": genre,
        "num_examples": n
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()


def parse_examples_into_list_of_paths(examples):
    return [example['filepath'] for example in examples]


def download_songs_zip(paths, zip_name, save_path):
    """
    Downloads a ZIP file containing the specified songs.

    :param paths: List of song paths to include in the ZIP file.
    :param zip_name: Name of the ZIP file to be created on the server.
    :param save_path: Path where the downloaded ZIP file should be saved.
    """
    # Create the JSON payload
    payload = {
        "paths": paths,
        "zip_name": zip_name
    }

    # Send the POST request
    url = f"{MEGAPI_URL}/minio/download-zip"
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {get_token()}',
        'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, json=payload)

    # Ensure the directory exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Save the response content to a file if the request was successful
    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
        print(f"File downloaded successfully: {save_path}")
    else:
        print("Failed to download file")
        print(response.text)

In [ ]:
ontology = [
    'blues',
    'chanson',
    'classical',
    'country',
    'dance',
    'dub',
    'electronic',
    'folk',
    'funk',
    'hard rock',
    'Hip-Hop',
    'house',
    'jazz',
    'metal',
    'pop',
    'rap',
    'reggae',
    'rock',
]

In [7]:
# Create the base directory for the dataset
base_dir = "DATASET/originals/"
os.makedirs(base_dir, exist_ok=True)

for genre in ontology:

    examples = get_n_examples_of_genre(genre, 30)
    if (len(examples) < 30):
        print(f"Genre {genre} only has {len(examples)} examples.")

    list_of_paths = parse_examples_into_list_of_paths(examples)

    zip_name = f"{genre}.zip"
    save_path = os.path.join(base_dir, zip_name)
    download_songs_zip(list_of_paths, zip_name, save_path)

Genre blues only has 20 examples.
File downloaded successfully: DATASET/originals/blues.zip
Genre chanson only has 0 examples.
File downloaded successfully: DATASET/originals/chanson.zip
Genre classical only has 16 examples.
File downloaded successfully: DATASET/originals/classical.zip
Genre country only has 4 examples.
File downloaded successfully: DATASET/originals/country.zip
Genre dance only has 4 examples.
File downloaded successfully: DATASET/originals/dance.zip
Genre dub only has 0 examples.
File downloaded successfully: DATASET/originals/dub.zip
Genre electro only has 0 examples.
File downloaded successfully: DATASET/originals/electro.zip
File downloaded successfully: DATASET/originals/folk.zip
Genre funk only has 3 examples.
File downloaded successfully: DATASET/originals/funk.zip
Genre hard rock only has 0 examples.
File downloaded successfully: DATASET/originals/hard rock.zip
Genre hip-hop only has 0 examples.
File downloaded successfully: DATASET/originals/hip-hop.zip
Genre